### Get titles from URLS

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Load your CSV file into the DataFrame
df = pd.read_csv('labels.csv')
df

,GLOBALEVENTID,SQLDATE,MonthYear,Year,Actor1Name,Actor1CountryCode,Actor2Name,Actor2CountryCode,EventCode,NumMentions,NumSources,NumArticles,AvgTone,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,SOURCEURL,Municipality code
0,467522672,20150916,201509,2015,COLOMBIA,COL,RADIO STATION,NaN,193,4,1,4,-3.030303,CO,CO19,4.23161,-73.5079,10864086,http://www.unmultimedia.org/radio/english/2015...,Villavicencio
1,465775735,20150911,201509,2015,GUNMAN,NaN,COLOMBIA,COL,193,6,2,6,-5.497685,CO,CO19,4.23161,-73.5079,10864086,http://www.nzherald.co.nz/world/news/article.c...,Villavicencio
2,459806479,20150821,201508,2015,COLOMBIAN,COL,NaN,NaN,186,14,6,14,-2.199586,CO,CO19,4.00000,-73.3667,-589367,http://globenewswire.com/news-release/2015/08/...,Villavicencio
3,520388626,20160312,201603,2016,COLOMBIA,COL,JOURNALIST,NaN,193,2,1,2,-8.240535,CO,CO19,4.23161,-73.5079,10864086,http://www.dailystar.co.uk/news/latest-news/50...,Villavicencio
4,496403031,20151222,201512,2015,GOVERNMENT,NaN,COLOMBIA,COL,190,1,1,1,-5.557300,CO,CO19,4.32900,-72.8381,-590973,http://www.offnews.info/verArticulo.php?conten...,Villavicencio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31056,954996616,20201118,202011,2020,HONDURAS,HND,NaN,NaN,192,8,1,8,-5.488621,CO,CO21,8.80389,-72.9536,29198,https://www.marshallindependent.com/news/inter...,Cúcuta
31057,955046947,20201118,202011,2020,PRESIDENT,NaN,NaN,NaN,190,3,1,3,-4.580153,CO,CO21,8.80389,-72.9536,29198,https://colombiareports.com/colombia-rescues-d...,Cúcuta
31058,956040249,20201124,202011,2020,TOURIST,NaN,NaN,NaN,193,2,1,2,-2.453271,CO,CO21,8.80389,-72.9536,29198,https://www.laprensalatina.com/iota-deals-anot...,Cúcuta
31059,962369859,20210102,202101,2021,CARRIER,NaN,NaN,NaN,190,5,1,5,-1.003344,CO,CO21,8.27556,-72.3958,230744,https://www.flightglobal.com/safety/avianca-a3...,Cúcuta


In [3]:
def get_article_text_and_title(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extracting text
        article_text = ' '.join([p.text for p in soup.find_all('p')])

        # Attempt to extract a title from common tags
        title_tag = soup.find(['h1', 'h2', 'h3']) or soup.title
        page_title = title_tag.text.strip() if title_tag else 'No Title Found'

        return article_text, page_title
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None, None

In [ ]:
# Apply the function and split the results into two columns
df[['ExtractedText', 'ContentTitle']] = df['SOURCEURL'].apply(
    lambda url: pd.Series(get_article_text_and_title(url))
)


Error fetching http://www.unmultimedia.org/radio/english/2015/09/news-in-brief-16-september-2015-pm/: HTTPSConnectionPool(host='www.unmultimedia.org', port=443): Max retries exceeded with url: /radio/english/2015/09/news-in-brief-16-september-2015-pm/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1131)')))
Error fetching http://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=11511505: 404 Client Error: Not Found for url: https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=11511505
Error fetching http://faustasblog.com/2015/12/colombia-plebiscite-is-a-farce/: 403 Client Error: Forbidden for url: https://faustasblog.com/2015/12/colombia-plebiscite-is-a-farce/
Error fetching http://brudirect.com/0-world/international-headlines/item/21654-4-killed-in-clash-between-colombian-army-rebels: 404 Client Error: Not Found for url: https://brudirect.com/0-world/inter

In [ ]:
df

In [ ]:
# Save the DataFrame to a new CSV file
df.to_csv('labels_full.csv', index=False)